# Iris Data : Multiclass logistic regression

In [49]:
path = r"C:\Users\chan\Desktop\dataset"

import pandas as pd 
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [50]:
iris = pd.read_csv(path+"\\iris.csv")

In [51]:
iris.head()

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [52]:
xdata = iris.iloc[:,:-1]
ydata = pd.get_dummies(iris.iloc[:,-1])

In [53]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
xdata = scaler.fit_transform(xdata)

In [54]:
from sklearn.model_selection import train_test_split

xTrn, xTst, yTrn, yTst = train_test_split(xdata, ydata,
                                         test_size=0.3,
                                         random_state=42)

In [55]:
w = tf.Variable(tf.random_normal([4,3]))
b = tf.Variable(tf.random_normal([3]))

x = tf.placeholder(tf.float32, shape=[None,4])
y = tf.placeholder(tf.float32, shape=[None,3])

In [56]:
hf = tf.nn.softmax(tf.matmul(x,w)+b)
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=hf, labels=y))
opt = tf.train.GradientDescentOptimizer(1).minimize(cost)

In [57]:
import tensorflow.python.keras.backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

In [58]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1001):
        _, cv = sess.run([opt, cost],
                        feed_dict={x:xTrn,y:yTrn})
        if epoch%200 == 0:
            print(epoch, cv)
    res = sess.run(hf, feed_dict={x:xTst, y:yTst})
    pred = sess.run(tf.argmax(res,1))
    print("분류결과 : ", pred)
    sess.close()

0 0.7814364
200 0.6165421
400 0.5986734
600 0.5904429
800 0.5861349
1000 0.58349043
분류결과 :  [1 0 2 1 2 0 1 2 1 2 2 0 0 0 0 1 2 2 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]


In [70]:
from sklearn.metrics import accuracy_score

yTst['result'] = yTst.iloc[:,0]*0 + yTst.iloc[:,1]*1 + yTst.iloc[:,2]*2
accuracy_score(pred, np.array(yTst['result']))

C:\Users\chan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0.9333333333333333

# Zoo Data

In [193]:
zoo = pd.read_csv(path + "\\data-04-zoo.csv")

In [194]:
zoo_info = zoo.iloc[:18]
zoos = zoo.iloc[18:]

In [195]:
zoos.columns = zoo_info.iloc[:,0].values[1:]

In [196]:
xdata = zoos.iloc[:,:-1]
ydata = zoos.iloc[:,-1]

In [197]:
xdata = xdata.astype(str)
ydata = ydata.astype(str)
xdata = pd.get_dummies(xdata)
ydata = pd.get_dummies(ydata)

In [198]:
print(xdata.shape, ydata.shape)

(101, 36) (101, 7)


In [199]:
xTrn, xTst, yTrn, yTst = train_test_split(xdata, ydata, test_size=0.3)

In [200]:
w = tf.Variable(tf.random_normal(shape=[xdata.shape[1], ydata.shape[1]]))
b = tf.Variable(tf.random_normal(shape=[ydata.shape[1]]))

In [201]:
x = tf.placeholder(tf.float32, shape=[None, xdata.shape[1]])
y = tf.placeholder(tf.float32, shape=[None, ydata.shape[1]])

In [202]:
hf = tf.nn.softmax(tf.matmul(x,w)+b)
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=hf, labels=y))

In [203]:
opt = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
pred = tf.argmax(hf, axis=1)
score = tf.equal(pred, tf.argmax(y, axis=1))

In [204]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(2001):
        _, loss_val = sess.run([opt, loss], feed_dict={x:xTrn, y:yTrn})
        if epoch%200 == 0:
            print(epoch, ':', loss_val)
    result, score_ = sess.run([pred,score], feed_dict={x:xTst, y:yTst})
    print(result, score_)
    sess.close()

0 : 0.7595306
200 : 0.743091
400 : 0.71754134
600 : 0.7042631
800 : 0.69785607
1000 : 0.69356835
1200 : 0.6907559
1400 : 0.6889559
1600 : 0.68764895
1800 : 0.6866074
2000 : 0.68572336
[1 0 0 2 0 2 5 1 2 5 3 0 1 1 0 0 1 2 0 0 0 0 0 0 5 1 0 1 2 1 2] [False False  True False  True False  True  True False  True False False
 False  True  True False  True False  True  True  True False  True  True
 False  True  True  True False False False]
